In [1]:
import numpy as np
import os
import json
import utils as utils

from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix

from ml_statistical_features import load_data, get_patient_split, get_all_scores, get_rf_grid_params, get_lda_grid_params, get_dt_grid_params, get_mlp_grid_params, get_linear_svc_grid_params, eval_classifier_paper
from notebook_md_utils import get_md_data_distribution_string, get_md_mean_accuracy_grid, get_md_test_accuracy_grid, get_md_confusion_matrix_grid

from IPython.display import Markdown, display

In [2]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

# Daten laden

In [3]:
x, y, patient_id = load_data(segment_length=10, overlap_amount=0, hr_threshold=10)

# Splitten der Daten in G1 und G2

## Labelverteilung

### Im Paper:
58% informativ

42% nicht-informativ


In [4]:
distribution = np.bincount(y)

display(Markdown("### Vorliegende Daten"))
display(Markdown("%i %s informativ" % (round(100/len(y)*distribution[1]), '%')))
display(Markdown("%i %s nicht-informativ" % (round(100/len(y)*distribution[0]), '%')))

### Vorliegende Daten

52 % informativ

48 % nicht-informativ


## Gruppenverteilung
Segmente zufällig in 2 Gruppen unterteilt
### Im Paper


|   | informativ | nicht-informativ | gesamt    |   
|:--|:--------- :|:----------------:|:---------:|
| G1 |1296 (62%)  | 789 (38%)        | 2085 (57%)|
| G2 |813 (53%)   | 733 (47%)        | 1546 (43%)|

In [5]:
x_g1, x_g2, y_g1, y_g2, groups1, groups2 = get_patient_split(x, y, test_size=0.33, patient_id=patient_id)

display(Markdown("### Vorliegende Daten"))
display(Markdown(get_md_data_distribution_string(y_g1, y_g2)))

### Vorliegende Daten

|   | informativ | nicht-informativ | gesamt    |
|:--|:--------- :|:----------------:|:---------:|
|G1 | 28461 (50%)  | 28397 (50%)| 56858 (66%)|
|G2 | 16090 (56%)  | 12763 (44%)| 28853 (34%)|


## TODO: Coverage + Mean Error

## 10-Fold cross validation mean accuracy for G1 and G2

### Im Paper

(RF: ntrees=50, SVM: rbf kernel, NN: 50 hidden neuron)

|    | RF    | SVM   | NN    | LDA   | DT    |
|:---|:-----:|:-----:|:-----:|:-----:|:-----:|
| G1 | 98.13 | 93.38 | 91.61 | 89.26 | 97.51 |
| G2 | 92.30 | 90.49 | 85.89 | 79.37 | 89.39 |

In [6]:
paths = ['LDA_hr10', 'DT_hr10', 'RF_hr10', 'MLP_hr10']
scores = get_all_scores(reconstruct=False, paths=paths)


display(Markdown("### Vorliegende Daten"))
display(Markdown(get_md_mean_accuracy_grid(scores)))

### Vorliegende Daten

| | LDA| DT| RF| MLP| 
|:--|:--:|:--:|:--:|:--:|
 | G1 |  51.79  |  53.97  |  56.66  |  56.31  | 
 | G2 |  45.99  |  50.67  |  48.31  |  53.69  | 

## Accuracy results for testing G2 vs. G1 (Exp1) and testing G1 vs. G2 (Exp2)

### Im Paper

|      | RF    | SVM   | NN    | LDA   | DT    |
|:-----|:-----:|:-----:|:-----:|:-----:|:-----:|
| Exp1 | 100   | 94.44 | 92.28 | 89.40 | 97.51 |
| Exp2 | 97.99 | 97.46 | 87.10 | 90.26 | 97.41 |
| Mean | 98.995| 95.95 | 89.69 | 89.83 | 98.41 |

In [7]:
display(Markdown("### Vorliegende Daten"))
display(Markdown(get_md_test_accuracy_grid(scores)))

### Vorliegende Daten

| | LDA| DT| RF| MLP| 
|:--|:--:|:--:|:--:|:--:|
 | Exp1 |  47.99  |  53.98  |  57.98  |  60.56  | 
 | Exp2 |  54.25  |  51.49  |  51.74  |  51.84  | 
 | Mean |  51.12 |  52.74 |  54.86 |  56.20 | 

## Confusion matrix of random forest for Exp2

### Im Paper

|             |                  | Actual        |                  |
|:------------|:----------------:|:-------------:|:----------------:|
|             |                  | informativ    | nicht-informativ |
|             |                  |               |                  |
|**Predicted**| informativ       | 1270          | 26               |
|             | nicht-informativ | 18            | 771              |

In [8]:
import pickle

path = os.path.join(utils.get_grid_params_path(), 'RF_hr10')
with open(os.path.join(path, 'fitted_model.sav'), 'rb') as file:
    rf = pickle.load(file)
    
_, x_test, _, y_test, _, _ = get_patient_split(x, y, test_size=0.33, patient_id=patient_id)
y_pred = rf.predict(x_test)

conf_mat = confusion_matrix(y_test, y_pred)

display(Markdown("### Vorliegende Daten"))
display(Markdown(get_md_confusion_matrix_grid(conf_mat)))

In [11]:
rf, _ = get_rf_grid_params()
_, _, _, _, _, y_pred_rf, y_true = eval_classifier_paper(x, y, patient_id, clf=rf, grid_folder_name='RF_hr10', patient_cv=True)
conf_mat = confusion_matrix(y_true, y_pred_rf)

display(Markdown("### Vorliegende Daten"))
display(Markdown(get_md_confusion_matrix_grid(conf_mat)))

### Vorliegende Daten

||| Actual ||
|:--|:--:|:--:|:--:|
||| informativ | nicht-informativ |
|||||
|**Predicted**| informativ | 10903 | 9684 |
|| nicht-informativ | 17558 | 18713 |

### Andere Klassifikatoren

In [12]:
lda, _ = get_lda_grid_params()
_, _, _, _, _, y_pred_lda, y_true = eval_classifier_paper(x, y, patient_id, clf=lda, grid_folder_name='LDA_hr10')
conf_mat = confusion_matrix(y_true, y_pred_lda)

display(Markdown("#### LDA"))
display(Markdown(get_md_confusion_matrix_grid(conf_mat)))

#### LDA

||| Actual ||
|:--|:--:|:--:|:--:|
||| informativ | nicht-informativ |
|||||
|**Predicted**| informativ | 23800 | 18615 |
|| nicht-informativ | 6049 | 8962 |

In [14]:
dt, _ = get_dt_grid_params()
_, _, _, _, _, y_pred_dt, y_true = eval_classifier_paper(x, y, patient_id, clf=dt, grid_folder_name='DT_hr10')
conf_mat = confusion_matrix(y_true, y_pred_dt)

display(Markdown("#### Decision Tree"))
display(Markdown(get_md_confusion_matrix_grid(conf_mat)))

#### Decision Tree

||| Actual ||
|:--|:--:|:--:|:--:|
||| informativ | nicht-informativ |
|||||
|**Predicted**| informativ | 18542 | 11116 |
|| nicht-informativ | 11307 | 16461 |

In [16]:
mlp, _ = get_mlp_grid_params()
_, _, _, _, _, y_pred_mlp, y_true = eval_classifier_paper(x, y, patient_id, clf=mlp, grid_folder_name='MLP_hr10')
conf_mat = confusion_matrix(y_true, y_pred_mlp)

display(Markdown("#### MLP"))
display(Markdown(get_md_confusion_matrix_grid(conf_mat)))

/Users/zianor/.virtualenvs/venv/Bachelorarbeit/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/zianor/.virtualenvs/venv/Bachelorarbeit/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/zianor/.virtualenvs/venv/Bachelorarbeit/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/zianor/.virtualenvs/venv/Bachelorarbeit/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWa

#### MLP

||| Actual ||
|:--|:--:|:--:|:--:|
||| informativ | nicht-informativ |
|||||
|**Predicted**| informativ | 18314 | 10137 |
|| nicht-informativ | 11535 | 17440 |

In [ ]:
svm, _ = get_linear_svc_grid_params()
_, _, _, _, _, y_pred_svm, y_true = eval_classifier_paper(x, y, patient_id, clf=svm, grid_folder_name='SVC_paper_hr10')
conf_mat = confusion_matrix(y_true, y_pred_svm)

display(Markdown("#### SVM"))
display(Markdown(get_md_confusion_matrix_grid(conf_mat)))

# Fehler nach RF Klassifizierung

In [ ]:
test_set = data.loc[y_true.index]
informativ_annotiert = test_set[test_set['informative_hr']]
non_informativ_annotiert = test_set[test_set['informative_hr'] == False]
non_informativ_annotiert = non_informativ_annotiert.replace([np.inf, -np.inf], np.nan)
informativ_rf = test_set[y_pred_rf]
informativ_rf = informativ_rf.replace([np.inf, -np.inf], np.nan)
non_informative_rf = test_set[y_pred_rf == False]
non_informative_rf = non_informative_rf.replace([np.inf, -np.inf], np.nan)

print("Relativer Fehler der Herzrate bei als informativ annotierten Segmenten %.2f %s" % (informativ_annotiert['rel_error'].mean(), "%"))
print("Absoluter Fehler der Herzrate bei als informativ annotierten Segmenten %.2f %s" % (informativ_annotiert['abs_error'].mean(), " bpm") )
print("Anzahl Segmente mit NaN Fehler bei als informativ vorausgesagten Segmenten %i" % (informativ_annotiert['abs_error'].isna().sum()))
print("Relativer Fehler der Herzrate bei als nicht informativ annotierten Segmenten %.2f %s" % (non_informativ_annotiert['rel_error'].mean(), "%"))
print("Absoluter Fehler der Herzrate bei als nicht informativ annotierten Segmenten %.2f %s" % (non_informativ_annotiert['abs_error'].mean(), " bpm") )
print("Anzahl Segmente mit NaN Fehler bei als nicht informativ annotierten Segmenten %i" % (non_informativ_annotiert['abs_error'].isna().sum()))

print("\n")

print("Relativer Fehler der Herzrate bei als informativ vorausgesagten Segmenten %.2f %s" % (informativ_rf['rel_error'].mean(), "%"))
print("Absoluter Fehler der Herzrate bei als informativ vorausgesagten Segmenten %.2f %s" % (informativ_rf['abs_error'].mean(), " bpm") )
print("Anzahl Segmente mit NaN Fehler bei als informativ vorausgesagten Segmenten %i" % (informativ_rf['abs_error'].isna().sum()))
print("Relativer Fehler der Herzrate bei als nicht informativ vorausgesagten Segmenten %.2f %s" % (non_informative_rf['rel_error'].mean(), "%"))
print("Absoluter Fehler der Herzrate bei als nicht informativ vorausgesagten Segmenten %.2f %s" % (non_informative_rf['abs_error'].mean(), " bpm") )
print("Anzahl Segmente mit NaN Fehler bei als nicht informativ vorausgesagten Segmenten %i" % (non_informative_rf['abs_error'].isna().sum()))